In [7]:
import pandas as pd
import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from time import sleep
from time import time
from datetime import datetime
from random import randint
from warnings import warn
import json
import pandas as pd
import urllib
from tqdm import tqdm


In [8]:
def wait_for_page_load(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = None
    while response is None:
        try:
            response = rq.get(url,headers=headers)
        except:
            sleep(3)
    return response

In [9]:
df = pd.read_csv("clean_url.csv")
print (df)

            date                                          url  \
0     2013-12-28  20131228102631/http://cointelegraph.com:80/   
1     2014-03-04  20140304062421/http://cointelegraph.com:80/   
2     2014-03-06  20140306224335/http://cointelegraph.com:80/   
3     2014-03-09  20140309142058/http://cointelegraph.com:80/   
4     2014-03-12  20140312015737/http://cointelegraph.com:80/   
...          ...                                          ...   
1717  2023-01-31    20230131235616/https://cointelegraph.com/   
1718  2023-02-01    20230201234017/https://cointelegraph.com/   
1719  2023-02-02    20230202232900/https://cointelegraph.com/   
1720  2023-02-03    20230203171716/https://cointelegraph.com/   
1721  2023-02-04    20230204132619/https://cointelegraph.com/   

                                               full_url  
0     https://web.archive.org/web/20131228102631/htt...  
1     https://web.archive.org/web/20140304062421/htt...  
2     https://web.archive.org/web/20140306224

In [10]:
tranchei = [100*i for i in (range (18))]
print(tranchei)

[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700]


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("titles_resumes_2014-07-07_to_2023-02-04.csv")

# Supprimer les lignes contenant des NA
df.dropna(inplace=True)

# Sauvegarder le dataframe modifié dans un nouveau fichier CSV
df.to_csv("titles_resumes_2014-07-07_to_2023-02-04B.csv", index=False)

In [19]:
news_title = []
news_summary = []
news_link = []
news_author = []
dates = []

    #pages = [str(i) for i in range(1,371)]

reqs = 0

start_time = time()

for url in tqdm(df['full_url'][30:1722]):
        try:
            pg = wait_for_page_load(url).text
            #print(pg)
        except urllib.error.HTTPError as e:
            print('Error: {}'.format(e))
            temp_data = {'title' : news_title, 'summary' : news_summary, 'link' : news_link}
            temp_df = pd.DataFrame(temp_data)
            temp_df.to_csv(f"titles_resumes_20first.csv", index=False)
        except urllib.error.URLError as e:
            print('Error: {}'.format(e.reason))
            temp_data = {'title' : news_title, 'summary' : news_summary, 'link' : news_link}
            temp_df = pd.DataFrame(temp_data)
            temp_df.to_csv(f"titles_resumes_20first.csv", index=False)
            
        sleep(randint(10,20))
        reqs +=1
        
        # Calculate elapsed time between requests
        elapsed_time = time() - start_time
        print('Request: {}; Frequency: {} requests/s'.format(reqs,reqs/elapsed_time))
 
        #Break once the max pages is reached
        if reqs > len(df['full_url']):
            warn('No. of requests was greater than expected')
            break
            
        # parse html using beautifulsoup and store in soup
        soup = bs(pg,'html.parser')
        
        #find all news posts
        posts = soup.find_all("div", class_="timeline-post col-md-8 col-sm-12 col-xs-24")
        print(posts)
        for post in posts:

            node = post.find("h2")
            if node.text != None:
                title = post.find("h2").text
                summary = post.find("a").text.strip()
            else:
                title = 'N/A'
                summary = 'N/A'
            news_title.append(title)
            news_summary.append(summary)
            news_link.append(url)
            dates.append(datetime.strptime(url[28:36], "%Y%m%d").strftime("%Y-%m-%d"))
        if len(posts) < 2:
            posts = soup.find_all("div", class_='col-xs-24 col-sm-12 col-md-8') # from 2015-03-13 to ? -- good
            for post in posts:
                #print (post)
                node2 = post.find("a")
                if node2 != None :
                    title = post.find("a").text.strip()
                else :
                    title = 'N/A'
                news_title.append(title) 
                news_link.append(url)
                dates.append(datetime.strptime(url[28:36], "%Y%m%d").strftime("%Y-%m-%d"))

            #<a href="https://web.archive.org/web/20170928082046/https://cointelegraph.com/news/blockchain-powers-shift-to-decentralization-in-media" title="Blockchain Powers Shift To Decentralization In Media"><span class="postTitle">Blockchain Powers Shift To Decentralization In Media</span></a>
        
        if len(posts) < 2:
            posts = soup.find_all("div", class_='col-xs-12 col-ssm-6 col-sm-6 col-md-4') # 2018-03-07 to -- good
            #<a href="https://web.archive.org/web/20180307044909/https://cointelegraph.com/news/steve-bannon-lauds-cryptocurrency-as-a-tool-to-resist-the-european-establishment" title="Steve Bannon Lauds Cryptocurrency As A Tool To Resist the ‘European Establishment’"> <span class="postTitle">Steve Bannon Lauds Cryptocurrency As A Tool To Resist the ‘European Establishment’</span> </a>
            for post in posts:
                node2 = post.find("a")
                if node2 != None :
                    title = post.find("a").text.strip()
                else :
                    title = 'N/A'
                news_title.append(title) 
                news_link.append(url)
                dates.append(datetime.strptime(url[28:36], "%Y%m%d").strftime("%Y-%m-%d"))

        if len(posts) < 2:
            posts = soup.find_all("li", class_='post-preview-list-cards__item') # 2019-04-10 -- good
            #<a class="post-preview-item-card__title" href="https://web.archive.org/web/20190410183429/https://cointelegraph.com/news/new-york-denies-bittrex-bitlicense-application-citing-inadequate-compliance" title="New York Denies Bittrex BitLicense Application Citing Inadequate Compliance"> New York Denies Bittrex BitLicense Application Citing Inadequate Compliance </a>
            for post in posts:
                #print(post)
                titles = soup.find_all('a', {'class': 'post-preview-item-card__title'})
                for t in titles:
                    if t.text.strip() not in news_title:
                        news_title.append(t.text.strip())
                        news_link.append(url)
                        dates.append(datetime.strptime(url[28:36], "%Y%m%d").strftime("%Y-%m-%d"))
        if len(posts) < 2:
            posts = soup.find_all("div", class_='post-card') # 2020-08-20 -- good
            #<a title="Crypto Exchange BtcTurk Becomes Main Sponsor of Turkey's National Soccer Teams" class="post-card__title-link"><span class="post-card__title"> Crypto Exchange BtcTurk Becomes Main Sponsor of Turkey's National Soccer Teams </span></a>
            for post in posts:
                node3 = post.find('span', class_='post-card__title')
                if node3 != None :
                    title = post.find('span', class_='post-card__title').text.strip()
                    
                else :
                    title = 'N/A'
                news_title.append(title)
                news_link.append(url)
                dates.append(datetime.strptime(url[28:36], "%Y%m%d").strftime("%Y-%m-%d")) 

if len(news_summary) == len(news_title):
    temp_data = {'dates': dates, 'title' : news_title, 'summary' : news_summary,'link' : url}
else:
    temp_data = {'dates': dates, 'title' : news_title,'link' : news_link}

#print(temp_data)
temp_df = pd.DataFrame(temp_data)
#print(temp_df)
filename = "titles_resumes_" + str(dates[0]) + "_to_" + str(dates[len(dates)-1]) + ".csv"
temp_df.to_csv(filename, index=False)

elapsed_time = time()
elapsed_time = elapsed_time - start_time
print('Elapsed time :')
print(elapsed_time)


  0%|          | 1/1692 [00:13<6:17:18, 13.39s/it]

Request: 1; Frequency: 0.07485149684558787 requests/s
[<div class="timeline-post col-md-8 col-sm-12 col-xs-24">
<div class="thumbnail" style="background-image: url('https://web.archive.org/web/20140707163654im_/http://cointelegraph.com/images/600_aHR0cDovL2NvaW50ZWxlZ3JhcGguY29tL3N0b3JhZ2UvdXBsb2Fkcy92aWV3LzI5ODI3MTI0ZjQ0ZWY0YmJmODEwNTczMTRkNjU4Y2UyLnBuZw==.jpg');">
<ul class="list-inline">
<li>2014-07-07 02:01 PM</li>
</ul>
<h2>How Businesses Can Start Accepting Bitcoins</h2>
<a href="/web/20140707163654/http://www.cointelegraph.com/news/112021/how-businesses-can-start-accepting-bitcoins">
                                                Businesses around the world are discovering that it pays to develop and employ a Bitcoin strategy.                                                <small style="display: block; margin-top: 10px;">
                                                    By Armand Tanzarian                                                </small>
</a>
</div>
</div>, <div class

  0%|          | 2/1692 [00:30<7:25:30, 15.82s/it]

Request: 2; Frequency: 0.06474430461984897 requests/s
[<div class="timeline-post col-md-8 col-sm-12 col-xs-24">
<div class="thumbnail" style="background-image: url('https://web.archive.org/web/20140711233035im_/http://cointelegraph.com/images/600_aHR0cDovL2NvaW50ZWxlZ3JhcGguY29tL3N0b3JhZ2UvdXBsb2Fkcy92aWV3L2FkOWJmOTE4NTU1Y2EzZjhmOGZkM2Q1MjEzMzRjNjQ1LnBuZw==.jpg');">
<ul class="list-inline">
<li>2014-07-11 04:15 PM</li>
</ul>
<h2>Top 5 Failures in Bitcoin History</h2>
<a href="/web/20140711233035/http://www.cointelegraph.com/news/112059/top-5-failures-in-bitcoin-history">
                                                A historical perspective for the Bitcoin community as it exists today.                                                <small style="display: block; margin-top: 10px;">
                                                    By Armand Tanzarian                                                </small>
</a>
</div>
</div>, <div class="timeline-post col-md-8 col-sm-12 col-xs-24">
<

  0%|          | 3/1692 [00:51<8:33:04, 18.23s/it]

...

Request: 1690; Frequency: 0.03550012315060045 requests/s
[]


100%|█████████▉| 1691/1692 [13:13:41<00:18, 18.60s/it]

Request: 1691; Frequency: 0.035509430450467824 requests/s
[]


100%|██████████| 1692/1692 [13:13:55<00:00, 28.15s/it]

Request: 1692; Frequency: 0.03552005436498916 requests/s
[]
Elapsed time :
47635.29781746864
